In [1]:
from polars_ds.extensions import StrExt, NumExt
import polars as pl
import numpy as np 

In [ ]:
df = pl.DataFrame({
    "a":["Atlanta"] * 100_000,
    "b":["Atlantis"] * 100_000,
})

df.head()

In [2]:
df = pl.DataFrame({
    "dummy": ["a"] * 50_000 + ["b"] * 50_000,
    "a": range(100_000),
    "b": range(-100_000, 0),
    "y": pl.Series(range(100_000, 200_000)) + 0.5
})

df.head()

dummy,a,b,y
str,i64,i64,f64
"""a""",0,-100000,100000.5
"""a""",1,-99999,100001.5
"""a""",2,-99998,100002.5
"""a""",3,-99997,100003.5
"""a""",4,-99996,100004.5


In [3]:
df.group_by("dummy").agg(
    pl.col("y").num_ext.lstsq([pl.col("a"), pl.col("b")], add_bias = True)
)

dummy,betas
str,list[f64]
"""a""","[0.668022, 0.331978, 133198.343417]"
"""b""","[-0.411356, 1.411356, 241136.143309]"


In [4]:
df.group_by("dummy").agg(
    pl.col("y").num_ext.lstsq([pl.col("a"), pl.col("b")], add_bias = False)
)

dummy,betas
str,list[f64]
"""a""","[2.000005, -1.000005]"
"""b""","[2.000005, -1.000005]"


In [6]:
%timeit df.select(pl.col("y").num_ext.lstsq([pl.col("a"), pl.col("b")]))

757 µs ± 19.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
df.select(
    pl.col("y").num_ext.lstsq([pl.col("a"), pl.col("b")]).over("dummy")
)

ComputeError: the length of the window expression did not match that of the group
> group: ("a")
> group length: 1
> output: 'shape: (2,)
Series: '' [f64]
[
	2.000005
	-1.000005
]'

Error originated in expression: 'col("y")./home/abstractqqq/Desktop/MY/Projects/polars_ds_extension/.venv/lib/python3.11/site-packages/polars_ds/_polars_ds.cpython-311-x86_64-linux-gnu.so:pl_lstsq([false.strict_cast(Boolean), col("a"), col("b")]).over([col("dummy")])'

In [ ]:
%timeit df.select(pl.col("y").num_ext.lstsq2(pl.col("a"), pl.col("b")))

In [ ]:
%timeit df.select(pl.col("y").num_ext.lstsq(pl.col("a"), pl.col("b")))

In [ ]:
%timeit df.select(pl.col("a").num_ext.gcd(15))
%timeit df.select(pl.col("a").num_ext.gcd(pl.col("b")))

In [ ]:
df.select(pl.col("a").num_ext.gcd2(pl.col("b")))